# 주식가격 하락으로 마음아픈 나를 위로해주는 챗봇 구현하기
 - 참조: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

## 1. import pacakages

In [ ]:
import random
import pandas as pd
import torch
import torch.nn as nn
from torch import optim
from tqdm import tqdm
torch.manual_seed(0) #to make the initial seeds
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #check whether GPU is available

In [ ]:
device

device(type='cuda')

## 2. Preparation of Datasets
 - 단어 표현을 위해 단어사전 만들기 단어 --> index, index --> 단어


In [ ]:
#!pip install Korpora
#from Korpora import Korpora
#Korpora.corpus_list()

In [ ]:
!git clone https://github.com/songys/Chatbot_data.git

Cloning into 'Chatbot_data'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 69 (delta 29), reused 5 (delta 3), pack-reused 18
Unpacking objects: 100% (69/69), 465.69 KiB | 3.37 MiB/s, done.


In [ ]:
raw = pd.read_csv("./Chatbot_data/ChatbotData.csv")

In [ ]:
raw

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [ ]:
qa_list = raw[["Q","A"]]
qa_list = qa_list.values.tolist()

In [ ]:
qa_list

[['12시 땡!', '하루가 또 가네요.'],
 ['1지망 학교 떨어졌어', '위로해 드립니다.'],
 ['3박4일 놀러가고 싶다', '여행은 언제나 좋죠.'],
 ['3박4일 정도 놀러가고 싶다', '여행은 언제나 좋죠.'],
 ['PPL 심하네', '눈살이 찌푸려지죠.'],
 ['SD카드 망가졌어', '다시 새로 사는 게 마음 편해요.'],
 ['SD카드 안돼', '다시 새로 사는 게 마음 편해요.'],
 ['SNS 맞팔 왜 안하지ㅠㅠ', '잘 모르고 있을 수도 있어요.'],
 ['SNS 시간낭비인 거 아는데 매일 하는 중', '시간을 정하고 해보세요.'],
 ['SNS 시간낭비인데 자꾸 보게됨', '시간을 정하고 해보세요.'],
 ['SNS보면 나만 빼고 다 행복해보여', '자랑하는 자리니까요.'],
 ['가끔 궁금해', '그 사람도 그럴 거예요.'],
 ['가끔 뭐하는지 궁금해', '그 사람도 그럴 거예요.'],
 ['가끔은 혼자인게 좋다', '혼자를 즐기세요.'],
 ['가난한 자의 설움', '돈은 다시 들어올 거예요.'],
 ['가만 있어도 땀난다', '땀을 식혀주세요.'],
 ['가상화폐 쫄딱 망함', '어서 잊고 새출발 하세요.'],
 ['가스불 켜고 나갔어', '빨리 집에 돌아가서 끄고 나오세요.'],
 ['가스불 켜놓고 나온거 같아', '빨리 집에 돌아가서 끄고 나오세요.'],
 ['가스비 너무 많이 나왔다.', '다음 달에는 더 절약해봐요.'],
 ['가스비 비싼데 감기 걸리겠어', '따뜻하게 사세요!'],
 ['가스비 장난 아님', '다음 달에는 더 절약해봐요.'],
 ['가장 확실한 건 뭘까?', '가장 확실한 시간은 오늘이에요. 어제와 내일을 놓고 고민하느라 시간을 낭비하지 마세요.'],
 ['가족 여행 가기로 했어', '온 가족이 모두 마음에 드는 곳으로 가보세요.'],
 ['가족 여행 고고', '온 가족이 모두 마음에 드는 곳으로 가보세요.'],
 ['가족 여행 어디로 가지?', '온 가족이 모두 마음에 드는 곳으로 가보세요.']

In [ ]:
#참조 모두의 딥러닝 시즌2
SOS_token = 0
EOS_token = 1
UNK_token = 2

# class for vocabulary related information of data
class Vocab:
    def __init__(self):
        self.vocab2index = {"<SOS>": SOS_token, "<EOS>": EOS_token, "<UNK>": UNK_token}
        self.index2vocab = {SOS_token: "<SOS>", EOS_token: "<EOS>", UNK_token: "<UNK>"}
        self.vocab_count = {}
        self.n_vocab = len(self.vocab2index)

    def add_vocab(self, sentence):
        for word in sentence.split(" "):
            if word not in self.vocab2index:
                self.vocab2index[word] = self.n_vocab
                self.vocab_count[word] = 1
                self.index2vocab[self.n_vocab] = word
                self.n_vocab += 1
            else:
                self.vocab_count[word] += 1


In [ ]:
vocab = Vocab()
vocab.add_vocab("안녕 나는 경태라고해")

In [ ]:
vocab.vocab2index["안녕"]

3

  - 데이터 전처리: 학습데이터로 만들기

In [ ]:
# read and preprocess the corpus data
def preprocess_df(corpus, source_max_length, target_max_length):
    print("reading corpus...")
    pairs = corpus

    source_vocab = Vocab()
    target_vocab = Vocab()

    print("Counting words...")
    for pair in pairs:
        source_vocab.add_vocab(pair[0])
        target_vocab.add_vocab(pair[1])
    print("source vocab size =", source_vocab.n_vocab)
    print("target vocab size =", target_vocab.n_vocab)

    return pairs, source_vocab, target_vocab

In [ ]:
SOURCE_MAX_LENGTH = 50
TARGET_MAX_LENGTH = 50
load_pairs, load_source_vocab, load_target_vocab = preprocess_df(qa_list, source_max_length=SOURCE_MAX_LENGTH, target_max_length=TARGET_MAX_LENGTH)

reading corpus...
Counting words...
source vocab size = 14287
target vocab size = 10008


In [ ]:
load_pairs[:3]

[['12시 땡!', '하루가 또 가네요.'],
 ['1지망 학교 떨어졌어', '위로해 드립니다.'],
 ['3박4일 놀러가고 싶다', '여행은 언제나 좋죠.']]

In [ ]:
load_source_vocab.vocab2index["위로해"]

12742

In [ ]:
load_target_vocab.vocab2index["위로해"]

6

In [ ]:
load_source_vocab.vocab2index["안녕"], load_source_vocab.index2vocab[2]

(4006, '<UNK>')

In [ ]:
load_source_vocab.n_vocab, load_target_vocab.n_vocab

(14287, 10008)

In [ ]:
# convert sentence to the index tensor
def tensorize(vocab, sentence):
    #indexes = [vocab.vocab2index[word] for word in sentence.split(" ")]
    indexes = [vocab.vocab2index[word] if word in vocab.vocab2index else UNK_token for word in sentence.split(" ")]
    indexes.append(vocab.vocab2index["<EOS>"])
    return torch.Tensor(indexes).long().to(device).view(-1, 1)

In [ ]:
a_sample_tensor = tensorize(load_source_vocab, "학원폭력 짜증나")
print(a_sample_tensor)

tensor([[6098],
        [ 918],
        [   1]], device='cuda:0')


## 3. Model Classes

### 3-1. Encoder

In [ ]:
# declare simple encoder
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, x, hidden):
        x = self.embedding(x).view(1, 1, -1)
        x, hidden = self.gru(x, hidden)
        return x, hidden

In [ ]:
enc_hidden_size = 100
encoder = Encoder(input_size=load_source_vocab.n_vocab, hidden_size=enc_hidden_size).to(device)

In [ ]:
encoder_hidden = torch.zeros([1, 1, encoder.hidden_size]).to(device)
print(encoder_hidden)
print(a_sample_tensor)
output1, hidden1 = encoder(a_sample_tensor[0], encoder_hidden) # run for each toekn by token to RNN
print(hidden1)
output2, hidden2 = encoder(a_sample_tensor[1], hidden1)
print(hidden2)
output3, hidden3 = encoder(a_sample_tensor[2], hidden2)

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0.]]], device='cuda:0')
tensor([[6098],
        [ 918],
        [   1]], device='cuda:0')
tensor([[[ 0.3027,  0.0276,  0.3830, -0.1275, -0.2927, -0.0715,  0.0278,
           0.0699, -0.4929, -0.1895,  0.0845,  0.0570,  0.2859, -0.0945,
           0.3549,  0.1776,  0.1076,  0.1257, -0.0482,  0.0327,  0.0187,
          -0.1991,  0.1754,  0.0267,  0.1869,  0.5635,  0.2375,  0.0902,
           0.3577, -0.0649,  0.1949, -0.1186,  0.1871, -0.3549,  0.2992,
          -0.0633,  0.2241,  0.2642, -0.0520, -0.0240,  0.3801,  0.0114,
           0.2030,  0.1438

### 3-2. Decoder

In [ ]:
# declare simple decoder
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x, hidden):
        x = self.embedding(x).view(1, 1, -1)
        x, hidden = self.gru(x, hidden)
        x = self.softmax(self.out(x[0]))
        return x, hidden

In [ ]:
dec_hidden_size = enc_hidden_size
decoder = Decoder(hidden_size=dec_hidden_size, output_size=load_target_vocab.n_vocab).to(device)

In [ ]:
decoder_input = torch.Tensor([[SOS_token]]).long().to(device)
decoder_hidden = hidden3
decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)

In [ ]:
decoder_hidden

tensor([[[ 0.4012,  0.0843,  0.6428,  0.4105,  0.1180, -0.1977, -0.1121,
          -0.3175, -0.2089,  0.1996,  0.0904, -0.3564, -0.6629,  0.3325,
           0.1378,  0.3012,  0.2945, -0.0903, -0.3399,  0.1905, -0.5663,
          -0.4423,  0.0757, -0.2217, -0.5183,  0.4503,  0.1059,  0.0792,
           0.4802,  0.1020,  0.2567, -0.1859,  0.3087, -0.4922,  0.1907,
          -0.1612,  0.1516, -0.1084, -0.0714, -0.2615,  0.0128, -0.1157,
          -0.3022, -0.4033, -0.1774, -0.0820,  0.1492,  0.4167,  0.1115,
          -0.1499, -0.3060, -0.5810,  0.2039, -0.0369,  0.1501,  0.2937,
          -0.5511, -0.0203, -0.1659,  0.1199,  0.0807, -0.2168,  0.1294,
           0.4152,  0.4177,  0.0491, -0.4047,  0.2320,  0.2929, -0.3211,
          -0.0795,  0.1223,  0.2872,  0.2340, -0.0474,  0.1392, -0.1090,
           0.2386, -0.1092,  0.2390,  0.0292, -0.0845,  0.1947, -0.1404,
          -0.0874,  0.1624, -0.3830,  0.2980,  0.0047, -0.4499, -0.2040,
           0.0720, -0.6584,  0.5845,  0.5468, -0.32

## 4. Train function

In [ ]:
# training seq2seq
def train(pairs, source_vocab, target_vocab, encoder, decoder, encoder_optimizer, decoder_optimizer, n_iter, print_every=1000, learning_rate=0.01):
    encoder.train()
    decoder.train()
    loss_total = 0

    training_batch = [random.choice(pairs) for _ in range(n_iter)]
    training_source = [tensorize(source_vocab, pair[0]) for pair in training_batch] #user input
    training_target = [tensorize(target_vocab, pair[1]) for pair in training_batch] #answer

    criterion = nn.NLLLoss()

    for i in tqdm(range(1, n_iter + 1)):
        source_tensor = training_source[i - 1]
        target_tensor = training_target[i - 1]
        encoder_hidden = torch.zeros([1, 1, encoder.hidden_size]).to(device)

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        source_length = source_tensor.size(0)
        target_length = target_tensor.size(0)

        loss = 0

        for enc_input in range(source_length):
            _, encoder_hidden = encoder(source_tensor[enc_input], encoder_hidden)

        decoder_input = torch.Tensor([[SOS_token]]).long().to(device)
        decoder_hidden = encoder_hidden # connect encoder output to decoder input

        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # teacher forcing

        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        loss_iter = loss.item() / target_length
        loss_total += loss_iter

        if i % print_every == 0:
            loss_avg = loss_total / print_every
            loss_total = 0
            print("[{} - {}%] loss = {:05.4f}".format(i, i / n_iter * 100, loss_avg))

In [ ]:
# insert given sentence to check the training
def evaluate(pairs, source_vocab, target_vocab, encoder, decoder, target_max_length, print_every=5):
    encoder.eval()
    decoder.eval()
    for idx, pair in enumerate(pairs):

        source_tensor = tensorize(source_vocab, pair[0])
        source_length = source_tensor.size()[0]
        encoder_hidden = torch.zeros([1, 1, encoder.hidden_size]).to(device)

        for ei in range(source_length):
            _, encoder_hidden = encoder(source_tensor[ei], encoder_hidden)

        #decoder_input = torch.Tensor([[SOS_token]], device=device).long()
        decoder_input = torch.Tensor([[SOS_token]]).long().to(device)
        decoder_hidden = encoder_hidden
        decoded_words = []

        for di in range(target_max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            _, top_index = decoder_output.data.topk(1)
            if top_index.item() == EOS_token:
                decoded_words.append("<EOS>")
                break
            else:
                decoded_words.append(target_vocab.index2vocab[top_index.item()])

            decoder_input = top_index.squeeze().detach()

        predict_words = decoded_words
        predict_sentence = " ".join(predict_words)
        if idx%print_every==0:
          print(">", pair[0]) 
          print("=", pair[1])
          print("<", predict_sentence)
          print("")

## 5. Main function

In [ ]:
enc_hidden_size = 100
dec_hidden_size = enc_hidden_size
encoder = Encoder(input_size=load_source_vocab.n_vocab, hidden_size=enc_hidden_size).to(device)
decoder = Decoder(hidden_size=dec_hidden_size, output_size=load_target_vocab.n_vocab).to(device)

learning_rate = 0.01
encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

In [ ]:
train(load_pairs, load_source_vocab, load_target_vocab, encoder, decoder, encoder_optimizer, decoder_optimizer, n_iter=len(load_pairs)*5, print_every=500)

tensor([12074], device='cuda:0')


In [ ]:
load_pairs[1]

['1지망 학교 떨어졌어', '위로해 드립니다.']

In [ ]:
# check the model with given data
evaluate(load_pairs, load_source_vocab, load_target_vocab, encoder, decoder, TARGET_MAX_LENGTH, print_every=5000)

> 12시 땡!
= 하루가 또 가네요.
< 저는 위기 조차 없네요. <EOS>

> 학원폭력 짜증나
= 학교 폭력은 범죄에요.
< 좋은 사람이라면 고마워할 거예요. <EOS>

> 사랑한다고 말해주면 뭐가 덧나나
= 사랑한다고 표현해달라고 말해보세요.
< 사랑은 유지하는 게 중요한데 대단하네요. <EOS>



In [ ]:
a_test_sample = [['무슨 개소리를 이렇게 장황하게해', '?']]

In [ ]:
evaluate(a_test_sample, load_source_vocab, load_target_vocab, encoder, decoder, TARGET_MAX_LENGTH)

> 무슨 개소리를 이렇게 장황하게해
= ?
< 잘 견뎌내고 있네요. <EOS>



## 실습0: seq2seq 모델의 모델 부분을 구현해 보세요!

## 실습1: Encoder 부분을 조금 더 효율적으로 구현할 수 없을까? (병렬처리....)

## 실습2: Word Embedding, Language Model 이 두개가 제일 중요하다며? 이 두개는 어떻게씀?? 
 - 사전학습된 Word Embedding을 쓰면 성능 올라 단어임베딩을 넣으면 성능이 올라가겠지? 대신 험난한 여정이...
 - Language Model을 어떻게 사전학습해서 이 모델에 사용할 수 있을까?? (다음시간에)
 